## Preprocessing
Random state and classification cut-off value has been changed in the optimized notebook to further improve model accuracy.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
warnings.filterwarnings("ignore")
import tensorflow as tf
import pandas as pd
import warnings

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

2024-07-08 20:25:18.078936: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()
print("Unique Column Count:")
print(unique_values)

Unique Column Count:
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [5]:
cutoff_value = 500 
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
cutoff_value = 800
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION','USE_CASE','INCOME_AMT', 'ORGANIZATION', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,STATUS_0,STATUS_1,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,True,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,True,True,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,True,True,False
3,6692,1,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,True,False
4,142590,1,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,True,True,False


In [10]:
# Split our preprocessed data into our features and target arrays (target)
y = application_df['IS_SUCCESSFUL'].values
y

array([1, 1, 0, ..., 0, 1, 0])

In [11]:
# Split our preprocessed data into our features and target arrays (features)
X = application_df.drop(['IS_SUCCESSFUL'], axis =1).values
X

array([[5000, False, True, ..., True, True, False],
       [108590, False, False, ..., True, True, False],
       [5000, False, False, ..., True, True, False],
       ...,
       [5000, False, False, ..., True, True, False],
       [5000, False, False, ..., True, True, False],
       [36500179, False, False, ..., True, True, False]], dtype=object)

In [12]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
first_hidden_nodes = 40
nn.add(Dense(first_hidden_nodes, input_dim=input_features, activation='relu'))

# Second hidden layer
second_hidden_nodes = 20
nn.add(Dense(second_hidden_nodes, activation='relu'))

# Output layer
output_nodes = 1
nn.add(Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                1800      
                                                                 
 dense_1 (Dense)             (None, 20)                820       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,641
Trainable params: 2,641
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 1s 423us/step - loss: 0.5741 - accuracy: 0.7176
Epoch 2/80
804/804 [==============================] - 0s 408us/step - loss: 0.5556 - accuracy: 0.7285
Epoch 3/80
804/804 [==============================] - 0s 524us/step - loss: 0.5525 - accuracy: 0.7300
Epoch 4/80
804/804 [==============================] - 0s 405us/step - loss: 0.5506 - accuracy: 0.7305
Epoch 5/80
804/804 [==============================] - 0s 412us/step - loss: 0.5496 - accuracy: 0.7303
Epoch 6/80
804/804 [==============================] - 0s 407us/step - loss: 0.5488 - accuracy: 0.7312
Epoch 7/80
804/804 [==============================] - 0s 422us/step - loss: 0.5475 - accuracy: 0.7318
Epoch 8/80
804/804 [==============================] - 0s 442us/step - loss: 0.5471 - accuracy: 0.7308
Epoch 9/80
804/804 [==============================] - 0s 436us/step - loss: 0.5465 - accuracy: 0.7322
Epoch 10/80
804/804 [==============================] - 0s 420us/step - loss: 0.546

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5599 - accuracy: 0.7297 - 175ms/epoch - 651us/step
Loss: 0.5599485635757446, Accuracy: 0.72967928647995


### 1st Optimization - Increase Neurons/Nodes

In [18]:
# Define the model # Increase nodes in first hidden layer to test for improvement
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
first_hidden_nodes = 60 # Increasing amount of nodes
nn.add(Dense(first_hidden_nodes, input_dim=input_features, activation='relu'))

# Second hidden layer
second_hidden_nodes = 30 # Increasing amount of nodes
nn.add(Dense(second_hidden_nodes, activation='relu'))

# Output layer
output_nodes = 1
nn.add(Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 60)                2700      
                                                                 
 dense_4 (Dense)             (None, 30)                1830      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,561
Trainable params: 4,561
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile optimized model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 1s 434us/step - loss: 0.5741 - accuracy: 0.7185
Epoch 2/80
804/804 [==============================] - 0s 421us/step - loss: 0.5560 - accuracy: 0.7283
Epoch 3/80
804/804 [==============================] - 0s 425us/step - loss: 0.5527 - accuracy: 0.7299
Epoch 4/80
804/804 [==============================] - 0s 426us/step - loss: 0.5507 - accuracy: 0.7310
Epoch 5/80
804/804 [==============================] - 0s 422us/step - loss: 0.5498 - accuracy: 0.7307
Epoch 6/80
804/804 [==============================] - 0s 424us/step - loss: 0.5488 - accuracy: 0.7311
Epoch 7/80
804/804 [==============================] - 0s 422us/step - loss: 0.5480 - accuracy: 0.7323
Epoch 8/80
804/804 [==============================] - 0s 531us/step - loss: 0.5469 - accuracy: 0.7308
Epoch 9/80
804/804 [==============================] - 0s 424us/step - loss: 0.5463 - accuracy: 0.7309
Epoch 10/80
804/804 [==============================] - 0s 421us/step - loss: 0.546

In [21]:
# Evaluate the optimized model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5585 - accuracy: 0.7294 - 142ms/epoch - 532us/step
Loss: 0.5584573745727539, Accuracy: 0.7294460535049438


### 2nd Optimization - Add Additional Hidden Layer(s)

In [22]:
# Define the model # Add a third hidden layer (using additional nodes)
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
first_hidden_nodes = 60
nn.add(Dense(first_hidden_nodes, input_dim=input_features, activation='relu'))

# Second hidden layer
second_hidden_nodes = 30
nn.add(Dense(second_hidden_nodes, activation='relu'))

# Third hidden layer
third_hidden_nodes = 10
nn.add(Dense(third_hidden_nodes, activation='relu'))

# Output layer
output_nodes = 1
nn.add(Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 60)                2700      
                                                                 
 dense_7 (Dense)             (None, 30)                1830      
                                                                 
 dense_8 (Dense)             (None, 10)                310       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,851
Trainable params: 4,851
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile optimized model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 1s 461us/step - loss: 0.5764 - accuracy: 0.7171
Epoch 2/80
804/804 [==============================] - 0s 450us/step - loss: 0.5554 - accuracy: 0.7291
Epoch 3/80
804/804 [==============================] - 0s 446us/step - loss: 0.5528 - accuracy: 0.7304
Epoch 4/80
804/804 [==============================] - 0s 451us/step - loss: 0.5501 - accuracy: 0.7310
Epoch 5/80
804/804 [==============================] - 0s 451us/step - loss: 0.5497 - accuracy: 0.7319
Epoch 6/80
804/804 [==============================] - 0s 469us/step - loss: 0.5481 - accuracy: 0.7330
Epoch 7/80
804/804 [==============================] - 0s 496us/step - loss: 0.5471 - accuracy: 0.7329
Epoch 8/80
804/804 [==============================] - 0s 478us/step - loss: 0.5463 - accuracy: 0.7344
Epoch 9/80
804/804 [==============================] - 0s 463us/step - loss: 0.5453 - accuracy: 0.7328
Epoch 10/80
804/804 [==============================] - 0s 463us/step - loss: 0.544

In [25]:
# Evaluate the optimized model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5640 - accuracy: 0.7298 - 193ms/epoch - 721us/step
Loss: 0.5639842748641968, Accuracy: 0.7297959327697754


### 3rd Optimization - Decrease Node Count for (All 3) Hidden Layers

In [26]:
# Define the model # Add a third hidden layer (using additional nodes)
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
first_hidden_nodes = 30
nn.add(Dense(first_hidden_nodes, input_dim=input_features, activation='relu'))

# Second hidden layer
second_hidden_nodes = 20
nn.add(Dense(second_hidden_nodes, activation='relu'))

# Third hidden layer
third_hidden_nodes = 5
nn.add(Dense(third_hidden_nodes, activation='relu'))

# Output layer
output_nodes = 1
nn.add(Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 30)                1350      
                                                                 
 dense_11 (Dense)            (None, 20)                620       
                                                                 
 dense_12 (Dense)            (None, 5)                 105       
                                                                 
 dense_13 (Dense)            (None, 1)                 6         
                                                                 
Total params: 2,081
Trainable params: 2,081
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile optimized model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 1s 445us/step - loss: 0.5880 - accuracy: 0.7079
Epoch 2/80
804/804 [==============================] - 0s 440us/step - loss: 0.5589 - accuracy: 0.7256
Epoch 3/80
804/804 [==============================] - 0s 438us/step - loss: 0.5542 - accuracy: 0.7287
Epoch 4/80
804/804 [==============================] - 0s 438us/step - loss: 0.5518 - accuracy: 0.7304
Epoch 5/80
804/804 [==============================] - 0s 434us/step - loss: 0.5517 - accuracy: 0.7288
Epoch 6/80
804/804 [==============================] - 0s 428us/step - loss: 0.5497 - accuracy: 0.7304
Epoch 7/80
804/804 [==============================] - 0s 444us/step - loss: 0.5498 - accuracy: 0.7312
Epoch 8/80
804/804 [==============================] - 0s 433us/step - loss: 0.5488 - accuracy: 0.7322
Epoch 9/80
804/804 [==============================] - 0s 435us/step - loss: 0.5479 - accuracy: 0.7313
Epoch 10/80
804/804 [==============================] - 0s 429us/step - loss: 0.547

In [29]:
# Evaluate the optimized model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5568 - accuracy: 0.7300 - 147ms/epoch - 549us/step
Loss: 0.5567678809165955, Accuracy: 0.7300291657447815


### Final Optimization - 2 Hidden Layers with Additional Neurons in 1st Hidden Layer

In [48]:
# Define the model # Add a third hidden layer (using additional nodes)
nn = tf.keras.models.Sequential()

# Number of input features
input_features = len(X_train_scaled[0])

# First hidden layer
first_hidden_nodes = 90
nn.add(Dense(first_hidden_nodes, input_dim=input_features, activation='relu'))

# Second hidden layer
second_hidden_nodes = 40
nn.add(Dense(second_hidden_nodes, activation='relu'))

# Output layer
output_nodes = 1
nn.add(Dense(output_nodes, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 90)                4050      
                                                                 
 dense_36 (Dense)            (None, 40)                3640      
                                                                 
 dense_37 (Dense)            (None, 1)                 41        
                                                                 
Total params: 7,731
Trainable params: 7,731
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile optimized model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [50]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 1s 427us/step - loss: 0.5689 - accuracy: 0.7233
Epoch 2/80
804/804 [==============================] - 0s 416us/step - loss: 0.5538 - accuracy: 0.7282
Epoch 3/80
804/804 [==============================] - 0s 417us/step - loss: 0.5506 - accuracy: 0.7311
Epoch 4/80
804/804 [==============================] - 0s 415us/step - loss: 0.5492 - accuracy: 0.7320
Epoch 5/80
804/804 [==============================] - 0s 415us/step - loss: 0.5477 - accuracy: 0.7347
Epoch 6/80
804/804 [==============================] - 0s 415us/step - loss: 0.5468 - accuracy: 0.7339
Epoch 7/80
804/804 [==============================] - 0s 418us/step - loss: 0.5463 - accuracy: 0.7318
Epoch 8/80
804/804 [==============================] - 0s 489us/step - loss: 0.5459 - accuracy: 0.7332
Epoch 9/80
804/804 [==============================] - 0s 416us/step - loss: 0.5455 - accuracy: 0.7335
Epoch 10/80
804/804 [==============================] - 0s 415us/step - loss: 0.544

In [51]:
# Evaluate the optimized model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5647 - accuracy: 0.7310 - 163ms/epoch - 607us/step
Loss: 0.5647318959236145, Accuracy: 0.7309620976448059


In [53]:
num_features = X_train.shape[1]
print(f"Number of input features: {num_features}")

Number of input features: 44


In [52]:
# Export our optimized model to HDF5 file
nn.save('HDF5/AlphabetSoupCharity_Optimization.h5')

Our model accuracy is 73.92%, which is below 75%. but further optimization of cut-off values, random state, hidden layers, and nodes can improve overall accuracy. Using trial and error, I achieved 73% accuracy by adding a third hidden layer and decreasing node count per layer. With two hidden layers and normalized distribution parameters (increased node counts), the model reached 73.10% accuracy. Testing various epochs showed that 80 epochs yielded the best and most consistent results.

To reach the target accuracy, we need more tests and optimization. Next steps include segmenting the data further and adjusting for rare occurrences or outliers. Creating new features from available data can improve insights into funding success. For instance, filtering by income classification versus ask amount and removing outliers can refine the data. Testing different cut-off thresholds and node counts will help identify the optimal configuration. With more time and trials, we can confidently select the best number of hidden layers and neurons.